In [3]:
import os
from zipfile import ZipFile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import glob
import collections

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

from keras.layers import Input, Lambda, Dense, Flatten,Dropout,Conv2D,MaxPooling2D, BatchNormalization
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [4]:
os.environ["KAGGLE_USERNAME"]="dhanavikram"
os.environ["KAGGLE_KEY"]="f7ebebfb57d7ec2c7857bc257c72e33e"
!kaggle datasets download -d shashwatwork/knee-osteoarthritis-dataset-with-severity

file_name = '/content/knee-osteoarthritis-dataset-with-severity.zip'
with ZipFile(file_name, 'r') as zip_file:
  zip_file.extractall()

 99% 201M/204M [00:09<00:00, 24.3MB/s]
100% 204M/204M [00:09<00:00, 22.0MB/s]


In [5]:
first_class = []

base_dir = os.getcwd()
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
val_dir = os.path.join(base_dir, 'val')

# Resizing images
classes = [0, 1, 2]
xdata = dict()
for klass in [0,1,2]:
    
    xdata[klass] = []
    class_dir = os.path.join(train_dir,str(klass))
    image_list =  os.listdir(class_dir)

    for sample in image_list:
        img = cv2.resize(cv2.imread(class_dir+'/'+sample),(100,100))
        xdata[klass].append(img)

In [6]:
X = np.array(xdata[0] + xdata[1] + xdata[2])
print(X.shape)
Y = np.array([0 for i in range(len(xdata[0]))] + [1 for i in range(len(xdata[1]))] + [2 for i in range(len(xdata[2]))])
print(Y.shape)

(4848, 100, 100, 3)
(4848,)


In [7]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.3, random_state=0, shuffle=True)

In [8]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes= np.unique(ytrain),
                                                  y= ytrain)
class_weights = dict(enumerate(class_weights))

In [9]:
np.unique(ytrain)

array([0, 1, 2])

In [10]:
def create_model():
  model=Sequential()
  model.add(Conv2D(64,(3,3),activation='relu',input_shape=(100,100,3)))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Conv2D(32,(3,3),activation='relu'))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Conv2D(64,(3,3),activation='relu'))
  model.add(Dropout(0.5))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Conv2D(32,(3,3),activation='relu'))
  model.add(Dropout(0.5))
  #pooling layer
  model.add(MaxPooling2D(2,2))
  model.add(BatchNormalization())
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(3, activation='softmax'))

  return model

In [11]:
cnn = create_model()
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(cnn.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 49, 49, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                        

In [12]:
my_callbacks = [EarlyStopping(monitor='val_accuracy',patience=20, verbose=2, restore_best_weights=True)]

In [13]:
cnn.fit(xtrain,ytrain,epochs=300,batch_size=64,validation_data=(xtest,ytest),callbacks=my_callbacks)


Epoch 1/300
54/54 [==============================] - 12s 43ms/step - loss: 1.2585 - accuracy: 0.4103 - val_loss: 1.0641 - val_accuracy: 0.4502
Epoch 2/300
54/54 [==============================] - 2s 30ms/step - loss: 1.1006 - accuracy: 0.4436 - val_loss: 1.0736 - val_accuracy: 0.3684
Epoch 3/300
54/54 [==============================] - 2s 29ms/step - loss: 1.0619 - accuracy: 0.4763 - val_loss: 1.0710 - val_accuracy: 0.4289
Epoch 4/300
54/54 [==============================] - 2s 29ms/step - loss: 1.0588 - accuracy: 0.4839 - val_loss: 1.0457 - val_accuracy: 0.4914
Epoch 5/300
54/54 [==============================] - 2s 33ms/step - loss: 1.0548 - accuracy: 0.4801 - val_loss: 1.0795 - val_accuracy: 0.3856
Epoch 6/300
54/54 [==============================] - 2s 30ms/step - loss: 1.0457 - accuracy: 0.4928 - val_loss: 1.0885 - val_accuracy: 0.3759
Epoch 7/300
54/54 [==============================] - 2s 29ms/step - loss: 1.0427 - accuracy: 0.4795 - val_loss: 1.1211 - val_accuracy: 0.3794
Epoch

In [14]:
cnn.evaluate(xtest,ytest)

46/46 [==============================] - 0s 7ms/step - loss: 1.0134 - accuracy: 0.5402


[1.013407588005066, 0.5402061939239502]